In [1]:
import json
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.ndimage import gaussian_filter1d

def count_reps_from_json(json_frames, save_dir, json_name):
    y_values = []

    for frame_data in json_frames:
        landmarks = frame_data["landmarks"]
        
        # Extract left and right ankle y-coordinates from landmarks
        left_ankle = next((lm for lm in landmarks if lm["name"] == "left_ankle"), None)
        right_ankle = next((lm for lm in landmarks if lm["name"] == "right_ankle"), None)

        if left_ankle and right_ankle:
            # Use normalized y values (assuming y is normalized 0-1 or pixel coords - adapt if needed)
            left_ankle_y = left_ankle["y"]
            right_ankle_y = right_ankle["y"]
            avg_y = (left_ankle_y + right_ankle_y) / 2
            y_values.append(avg_y)

    y_values = np.array(y_values)
    y_inverted = 1 - y_values  # Invert for peak detection
    smoothed = gaussian_filter1d(y_inverted, sigma=3)

    peaks, _ = find_peaks(smoothed, distance=10)
    rep_count = len(peaks)
    print(f"{json_name}: {rep_count} reps")

    plt.figure(figsize=(10, 4))
    plt.plot(smoothed, label="Smoothed Ankle Y")
    plt.plot(peaks, smoothed[peaks], "ro", label="Reps")
    plt.title(f"Calf Raises Rep Count: {rep_count}")
    plt.xlabel("Frame")
    plt.ylabel("Inverted Y")
    plt.legend()
    plt.tight_layout()

    plot_path = os.path.join(save_dir, json_name.replace('.json', '_plot.png'))
    plt.savefig(plot_path)
    plt.close()

    return rep_count

# --- Example usage ---
input_folder = r"C:\Users\bdsid\OneDrive\Desktop\Dune Tech\Rep Counting\Data\Test_Bhaskar_Sir\calf_raises\Json"
output_folder = r"C:\Users\bdsid\OneDrive\Desktop\Dune Tech\Rep Counting\Data\Test_Bhaskar_Sir\calf_raises\Json"
os.makedirs(output_folder, exist_ok=True)

results = []
for file in os.listdir(input_folder):
    if file.endswith(".json"):
        with open(os.path.join(input_folder, file), 'r') as f:
            json_frames = json.load(f)
        reps = count_reps_from_json(json_frames, output_folder, file)
        results.append({"file": file, "reps": reps})

pd.DataFrame(results).to_csv(os.path.join(output_folder, "calf_rep_counts.csv"), index=False)


Calf_Raises_segment_5_start_09m01s870ms_pose.json: 19 reps
